In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

10. Сколько банок крабового мяса всего было заказано.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)